In [ ]:
from google.colab import files
from google.colab import auth
from google.colab import drive
auth.authenticate_user()
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pip

import pip

for each in ["requests","beautifulsoup4"]: 
    pip.main(['install', each])

import requests
from bs4 import BeautifulSoup


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#scraping indeed for positions according to candidate profile
import datetime, time
!pip install wget
import wget

import csv, re
import logging

import pandas as pd
import sqlite3
import json
from pytz import timezone

from requests.auth import HTTPBasicAuth
import requests
from requests.exceptions import HTTPError
import traceback


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=a9e31dbfa5af116e92b5058b1c74b8ca3722c9bb922671efe0948347f270a1cf
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
NumExpr defaulting to 2 threads.


In [ ]:
client_key = "77zk7frrida7k3"
client_secret = "CCISHQyTUjP3xqvZ"

In [ ]:
job_positions = ["data%20scientist%20intern","software%20engineering%20intern","Devops%20intern","Software%20Engineer","Data%20Analyst","Data%20Engineer"]
headers = {'Accept': 'application/json'}
auth = HTTPBasicAuth(client_key, client_secret)
job_url_template = "https://www.linkedin.com/jobs/view/"
job_df = pd.DataFrame(columns=["job_title","industry","company","description","job_locality","job_region","skills","job_post_expiry","education_requirements", "base_salary_min","base_salary_max","base_salary_currency","url"])

In [ ]:
list_of_links = []
try:
    s = requests.Session()
  
    for position in job_positions:
        for i in range(0,500,25):
        #for i in range(0,1):
            linkedin_url = 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords='+position+\
                            '&geoId=103644278&trk=public_jobs_jobs-search-bar_search-submit&position=8&pageNum=0&currentJobId=3257884433&start='+str(i)

            time.sleep(1)
            html_text = s.get(linkedin_url, headers=headers).text
            soup = BeautifulSoup(html_text, "html.parser")
            count=0
            for meat in soup.find_all('a'):
                if (meat.get('href') != None):
                    if(job_url_template in meat.get('href')):
                        list_of_links.append(meat.get('href'))
                        count+=1
    print(len(list_of_links))
except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')
except Exception as err:
    print(f'Other error occurred: {err}')

2997


In [ ]:
job_data = []
print(len(list_of_links))
for i in range(0,len(list_of_links)):
    job_req = requests.Session()
    job_url=list_of_links[i]
    try:
        job_text = job_req.get(job_url, headers=headers, auth=auth).text
        job_soup = BeautifulSoup(job_text, "html.parser")
        job_desc_soup = BeautifulSoup(job_text, "lxml")
        job_details_content = job_soup.find('script',  {"type": "application/ld+json"})
        job_desc = job_desc_soup.find('div', class_='description__text')

        if(job_details_content and job_details_content.text):
            job_json = json.loads(job_details_content.text)
            job_details = [job_json.get('title'), job_json.get('industry')]

            if(job_json.get('identifier')):
                job_details.append(job_json.get('identifier').get('name'))
            else:
                job_details.append(None)
            
            if(job_desc):
                job_details.append(job_desc.text.replace("Show more"," ").replace("Show less", " "))
            else:
                job_details.append(None)
            
            if('jobLocation' in job_json and 'address' in job_json.get('jobLocation')):
                job_details.extend( [job_json.get('jobLocation').get('address').get('addressLocality'), 
                                    job_json.get('jobLocation').get('address').get('addressRegion')])
            else:
                job_details.extend([None,None])
                
            job_details.extend( [job_json.get('skills'), job_json.get('validThrough')])

            if('educationRequirements' in job_json):
                job_details.append(job_json.get('educationRequirements').get('credentialCategory'))
            else:
                job_details.append(None)

            if('baseSalary' in job_json and 'value' in job_json.get('baseSalary')):
                job_details.extend([job_json.get('baseSalary').get('value').get('minValue'),
                          job_json.get('baseSalary').get('value').get('maxValue'), job_json.get('baseSalary').get('currency')])
            else:
                job_details.extend([None,None,None])
            job_details.append(job_url)
            
            job_data.append(job_details)
            time.sleep(1.25)
        else:
            print(job_url)
    except Exception as e:
        logging.error(traceback.format_exc())

2997
https://www.linkedin.com/jobs/view/data-scientist-at-pomeroy-3384652177?refId=rB2AOc0hyuwFqg3vO5QRPg%3D%3D&trackingId=0hQTD%2B%2FqSv3Calpxxgr9nQ%3D%3D&position=25&pageNum=0&trk=public_jobs_jserp-result_search-card
https://www.linkedin.com/jobs/view/data-scientist-full-time-at-bardess-group-ltd-3389875542?refId=BV5TqViF4IDtWAggaunTqw%3D%3D&trackingId=S6Vpsp5biV%2Bfvw8QmNONaA%3D%3D&position=4&pageNum=1&trk=public_jobs_jserp-result_search-card
https://www.linkedin.com/jobs/view/data-scientist-at-r-d-partners-3384656781?refId=BV5TqViF4IDtWAggaunTqw%3D%3D&trackingId=Waevn1%2BG%2Bfxv3jqeoNrv7A%3D%3D&position=10&pageNum=1&trk=public_jobs_jserp-result_search-card
https://www.linkedin.com/jobs/view/data-scientist-at-hireresources-3384151389?refId=BV5TqViF4IDtWAggaunTqw%3D%3D&trackingId=0Sn6O4jMee9fSZ8ymgl8dQ%3D%3D&position=22&pageNum=1&trk=public_jobs_jserp-result_search-card
https://www.linkedin.com/jobs/view/python-data-scientist-at-brains-workgroup-3395566644?refId=P85ov0M%2B4Tnt8o8jAr4

ERROR: Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py", line 597, in urlopen
    httplib_response = self._make_request(conn, method, url,
  File "/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py", line 343, in _make_request
    self._validate_conn(conn)
  File "/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py", line 839, in _validate_conn
    conn.connect()
  File "/usr/local/lib/python3.8/dist-packages/urllib3/connection.py", line 337, in connect
    self.sock = ssl_wrap_socket(
  File "/usr/local/lib/python3.8/dist-packages/urllib3/util/ssl_.py", line 345, in ssl_wrap_socket
    return context.wrap_socket(sock, server_hostname=server_hostname)
  File "/usr/lib/python3.8/ssl.py", line 500, in wrap_socket
    return self.sslsocket_class._create(
  File "/usr/lib/python3.8/ssl.py", line 1040, in _create
    self.do_handshake()
  File "/usr/lib/python3.8/ssl.py", line 1309, in do_handshake
    se

https://www.linkedin.com/jobs/view/data-engineer-at-proven-recruiting-3373349198?refId=UUJhKGHjUE4xrIzLGhR9Jg%3D%3D&trackingId=XdKEQKi%2BMqOVb%2FvSwmlDFA%3D%3D&position=8&pageNum=11&trk=public_jobs_jserp-result_search-card
https://www.linkedin.com/jobs/view/azure-data-engineer-at-techpeople-usa-3387563418?refId=1JIVG7OlqL74yicIQP30hA%3D%3D&trackingId=si7FJ%2Fmlx%2FYwhdDLDdqpRw%3D%3D&position=8&pageNum=12&trk=public_jobs_jserp-result_search-card
https://www.linkedin.com/jobs/view/data-engineer-at-flex-fleet-rental-3385814614?refId=1JIVG7OlqL74yicIQP30hA%3D%3D&trackingId=fb8hRAcTdRgcdGte5amwMw%3D%3D&position=14&pageNum=12&trk=public_jobs_jserp-result_search-card
https://www.linkedin.com/jobs/view/%09data-engineer-at-hays-3381689405?refId=OamvyHJETh2E%2FtYX1EpqUA%3D%3D&trackingId=PbhjjpTIMsaPHDIbqvFJ5Q%3D%3D&position=19&pageNum=18&trk=public_jobs_jserp-result_search-card


In [ ]:
job_df = pd.DataFrame(job_data, 
                       columns=["job_title","industry","company","description", "job_locality","job_region","skills","job_post_expiry","education_requirements", "base_salary_min","base_salary_max","base_salary_currency","url"])

job_df

,job_title,industry,company,description,job_locality,job_region,skills,job_post_expiry,education_requirements,base_salary_min,base_salary_max,base_salary_currency,url
0,Data Scientist Internship - OR (US),Insurance,Asurion,\n\n\n 2022-75202Asurion Data Science I...,Nashville,TN,,2023-01-04T03:56:32.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-scient...
1,Intern - Data Scientist,Medical Equipment Manufacturing,Dexcom,"\n\n\nAbout DexcomDexcom, Inc. empowers people...",United States,None,,2023-01-08T04:50:37.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/intern-data...
2,"Data Science, Graduate Internship",Retail,"Lowe's Companies, Inc.",\n\n\nLowe’s Summer Internship Program Overvie...,Charlotte,NC,,2023-01-09T14:48:39.000Z,None,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-scienc...
3,Data Scientist Internship - ML (US),Insurance,Asurion,\n\n\n 2022-75205Asurion Data Science I...,Nashville,TN,,2023-01-04T03:56:32.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-scient...
4,Data Science Intern,Manufacturing,Nestlé Purina North America,\n\n\n We’re a leader in the pet care i...,St Louis,MO,,2023-01-05T20:57:11.000Z,None,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-scienc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2930,"Data Engineer (AWS, Python, SQL)",Staffing and Recruiting,"Cambay Healthcare, LLC",\n\n\nDescriptionData EngineerWe are currently...,Durham,NC,,2023-01-09T17:59:24.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-engine...
2931,Data Engineer,Software Development,Diverse Lynx,\n\n\nJob DescriptionPerform Data Engineering ...,United States,None,,2023-01-03T06:44:43.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-engine...
2932,Data Engineer,Software Development,Diverse Lynx,\n\n\nJob DescriptionData model development an...,Chandler,AZ,,2023-01-03T08:59:18.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-engine...
2933,Associate Data Engineer,IT Services and IT Consulting,Neudesic,\n\n\nAbout NeudesicPassion for technology dri...,Irvine,CA,,2023-01-01T17:53:58.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/associate-d...


In [ ]:
job_df['description']

0       \n\n\n        2022-75202Asurion Data Science I...
1       \n\n\nAbout DexcomDexcom, Inc. empowers people...
2       \n\n\nLowe’s Summer Internship Program Overvie...
3       \n\n\n        2022-75205Asurion Data Science I...
4       \n\n\n        We’re a leader in the pet care i...
                              ...                        
2930    \n\n\nDescriptionData EngineerWe are currently...
2931    \n\n\nJob DescriptionPerform Data Engineering ...
2932    \n\n\nJob DescriptionData model development an...
2933    \n\n\nAbout NeudesicPassion for technology dri...
2934    \n\n\n        Evaluating existing data systems...
Name: description, Length: 2935, dtype: object

In [ ]:
print(job_data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
job_df = pd.DataFrame(job_data, 
                       columns=["job_title","industry","company","description", "job_locality","job_region","skills","job_post_expiry","education_requirements", "base_salary_min","base_salary_max","base_salary_currency","url"])
outfile="jobs_data"
outfile=outfile+'.csv'
print(outfile)
job_df.to_csv(outfile, sep=',', encoding='utf-8')

job_df

jobs_data.csv


,job_title,industry,company,description,job_locality,job_region,skills,job_post_expiry,education_requirements,base_salary_min,base_salary_max,base_salary_currency,url
0,Data Scientist Internship - OR (US),Insurance,Asurion,\n\n\n 2022-75202Asurion Data Science I...,Nashville,TN,,2023-01-04T03:56:32.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-scient...
1,Intern - Data Scientist,Medical Equipment Manufacturing,Dexcom,"\n\n\nAbout DexcomDexcom, Inc. empowers people...",United States,None,,2023-01-08T04:50:37.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/intern-data...
2,"Data Science, Graduate Internship",Retail,"Lowe's Companies, Inc.",\n\n\nLowe’s Summer Internship Program Overvie...,Charlotte,NC,,2023-01-09T14:48:39.000Z,None,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-scienc...
3,Data Scientist Internship - ML (US),Insurance,Asurion,\n\n\n 2022-75205Asurion Data Science I...,Nashville,TN,,2023-01-04T03:56:32.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-scient...
4,Data Science Intern,Manufacturing,Nestlé Purina North America,\n\n\n We’re a leader in the pet care i...,St Louis,MO,,2023-01-05T20:57:11.000Z,None,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-scienc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2930,"Data Engineer (AWS, Python, SQL)",Staffing and Recruiting,"Cambay Healthcare, LLC",\n\n\nDescriptionData EngineerWe are currently...,Durham,NC,,2023-01-09T17:59:24.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-engine...
2931,Data Engineer,Software Development,Diverse Lynx,\n\n\nJob DescriptionPerform Data Engineering ...,United States,None,,2023-01-03T06:44:43.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-engine...
2932,Data Engineer,Software Development,Diverse Lynx,\n\n\nJob DescriptionData model development an...,Chandler,AZ,,2023-01-03T08:59:18.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/data-engine...
2933,Associate Data Engineer,IT Services and IT Consulting,Neudesic,\n\n\nAbout NeudesicPassion for technology dri...,Irvine,CA,,2023-01-01T17:53:58.000Z,bachelor degree,NaN,NaN,None,https://www.linkedin.com/jobs/view/associate-d...


In [ ]:
!cp "/content/jobs_data.csv" "/content/drive/MyDrive/jobs_data.csv"
print("Uploaded")

Uploaded
